# **Step 02: Model Scoring**

# A. Create *Scoring Script* for Azure ML Deployment

In [ ]:
import os

# create src folder
os.makedirs("src", exist_ok=True)

In [ ]:
%%writefile src/scoring.py
# scoring.py
# Inference script for Azure ML endpoints
# Exposes init() to load the model and run() to process inference requests

import json
import base64
import io
import os
import glob
import logging
from PIL import Image
import torch
from torchvision import transforms, models


def init():
    # Globals that will be reused across endpoint calls
    global model, transform, class_names

    # Azure ML provides the model directory via environment variable
    model_dir = os.getenv("AZUREML_MODEL_DIR")
    assert model_dir and os.path.exists(model_dir), f"AZUREML_MODEL_DIR not available: {model_dir}"

    # Resolve model file path inside the mounted folder
    model_path = os.path.join(model_dir, "model.pth")
    if not os.path.exists(model_path):
        # In case model.pth is nested inside version folders
        matches = glob.glob(os.path.join(model_dir, "**", "model.pth"), recursive=True)
        assert matches, f"model.pth not found under {model_dir}"
        model_path = matches[0]

    # Load checkpoint from training
    ckpt = torch.load(model_path, map_location="cpu")

    # Extract class names and number of classes
    class_names = ckpt["classes"]
    num_classes = len(class_names)

    # Load EfficientNet-B0 backbone with pretrained weights
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

    # Replace final classifier layer to match training configuration
    model.classifier = torch.nn.Linear(model.classifier[1].in_features, num_classes)

    # Load trained weights
    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()  # Set model to evaluation mode

    # Preprocessing transform (same as training)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

def run(raw_data):
    # Handle a single inference request from Azure ML
    try:
        data = json.loads(raw_data)

        # Expect a base64 encoded image
        img_b64 = data.get("image", None)
        if img_b64 is None:
            return {"error": "No image provided"}

        # Decode image and convert to a PIL Image
        img_bytes = base64.b64decode(img_b64)
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

        # Apply preprocessing and add batch dimension
        x = transform(img).unsqueeze(0)  # [1, 3, 224, 224]

        # Perform forward pass
        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, dim=1).squeeze().tolist()
            pred_idx = int(torch.argmax(logits, dim=1).item())
            pred_label = class_names[pred_idx]

        # Return prediction and probability distribution
        return {
            "prediction": pred_label,
            "probabilities": probs
        }

    except Exception as e:
        # Return fallback error message
        return {"error": str(e)}

# B. Run Offline Inference using scoring.py

In [ ]:
import json
import base64
import os
from pathlib import Path

import sys
sys.path.append("src")

# Set environment variable so scoring.py can locate model.pth locally
os.environ["AZUREML_MODEL_DIR"] = str(Path("artifacts").resolve())

# Import the scoring module written earlier
import scoring

# Initialize the model and preprocessing pipeline
scoring.init()

# Load a sample test image from the dataset
with open("leaf-disease-dataset/test_healthy.jpg", "rb") as f:
    img_bytes = f.read()

# Encode the image as base64 (same format used by Azure ML endpoint requests)
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

# Construct the raw JSON string expected by scoring.run()
raw_data = json.dumps({"image": img_b64})

# Run inference using the scoring.py logic
result = scoring.run(raw_data)
print(result)